In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
def get_page():
    docs = []
    for i in range(1, 3):
        url = 'https://www.eatthismuch.com/food/browse/?q=indian&type=branded&page=' + str(i) + '&show_nutrient=fiber&nutrition_display=serving_'
        response = requests.get(url)
        if response.status_code != 200:
            print('Status code:', response.status_code)
            raise Exception('Failed to fetch web page ' + url)
        doc = BeautifulSoup(response.text)
        docs.append(doc)
    
    return docs

In [3]:
def process_it(samp):
    for i in range(len(samp)):
        samp[i] = samp[i].replace(' ', '')
        samp[i] = samp[i].replace('g', '')
        if '\n' in samp[i]:
            samp[i] = samp[i].replace('\n', '')
    return samp

In [5]:
def get_df(docs):
    food_names = []
    calories = []
    Carbs = []
    fats = []
    proteins = []
    fibers = []
    
    for doc in docs:
        food = doc.find_all('div', {'class': 'result_name col-3'})
        cal = doc.find_all('div', {'class': 'col-2 offset-1 nutrient_cell'})
        nutri = doc.find_all('div', {'class': 'col-2 nutrient_cell'})
        carbs = nutri[::3]
        fat = nutri[1::3]
        protein = nutri[2::3]
        fiber = doc.find_all('div', {'class': 'col-3 nutrient_cell'})
        
        for i in food:
            food_names.append(i.text.strip())
            
        for i in cal:
            calories.append(i.text.strip())
            
        for i in carbs:
            Carbs.append(i.text.strip())
            Carbs = process_it(Carbs)
            
        for i in fat:
            fats.append(i.text.strip())
            fats = process_it(fats)
            
        for i in protein:
            proteins.append(i.text.strip())
            proteins = process_it(proteins)
            
        for i in fiber:
            fibers.append(i.text.strip())
            fibers = process_it(fibers)
        
    food_df = pd.DataFrame({'Name': food_names
                            , 'Calories': calories
                            , 'Carbs': Carbs
                            , 'Fats': fats
                            , 'Proteins': proteins
                            , 'Fiber': fibers})
        
    return food_df

In [14]:
pages = get_page()

In [15]:
len(pages)

16

In [16]:
df1 = get_df(pages)

In [17]:
len(df1)

313

### For restaurant section

In [25]:
def get_res_page():
    docs = []
    for i in range(1, 3):
        url = 'https://www.eatthismuch.com/food/browse/?q=indian&type=restaurant&page=' + str(i) + '&show_nutrient=fiber&nutrition_display=serving_'
        response = requests.get(url)
        if response.status_code != 200:
            print('Status code:', response.status_code)
            raise Exception('Failed to fetch web page ' + url)
        doc = BeautifulSoup(response.text)
        docs.append(doc)
    
    return docs

In [26]:
p = get_res_page()

In [27]:
dfr = get_df(pages)

In [28]:
dfr.head()

,Name,Calories,Carbs,Fats,Proteins,Fiber
0,"Butter Chicken, Indian Stylee, Frozen",280,43.0,4.5,17,2
1,"Indian Cuisine, Punjab Potatoes",121,16,5,4,5
2,"Fit Popcorn Chips, Spinach Feta",130,19.0,5.0,2,1
3,Aged White Cheddar Popcorn,150,14.0,9.0,2,2
4,"Indian Snacks, Boondi, Chick Peas Balls",290,22,20,6,1
